# Sudoku assistant

<div class="alert alert-block alert-info">
    &#9432; This notebook (<a href="https://github.com/opvious/notebooks/blob/main/resources/examples/sudoku.ipynb">source</a>) can be executed directly from your browser when accessed via its <a href="https://www.opvious.io/notebooks/retro/notebooks/?path=examples/sudoku.ipynb">opvious.io/notebooks</a> URL.
</div>

This notebook shows how we can use the [Opvious Python SDK](https://github.com/opvious/sdk.py) to solve Sudoku grids and identify mistakes. You will not only be able to quickly find solutions but also restore feasibility in grids such as this one:

<div align="center"><a href="https://www.reddit.com/r/sudoku/comments/2xbksc/where_did_i_go_wrong_help_me_please/"><img src="https://imgur.com/0VAdsys.png" style="height: 500px; margin: 2em;"/></a></div>




In [1]:
%pip install opvious

## Setup

We first introduce a few utilities to parse and render Sudoku grids.

In [2]:
import pandas as pd

def parse_grid(s):
    """Parses (row, column, value) triples from a grid's string representation"""
    return [
        (i, j, int(c))
        for i, line in enumerate(s.strip().split("\n"))
        for j, c in enumerate(line.strip())
        if c != "."
    ]

def pretty_grid(triples):
    """Pretty-prints a list of triples as a 2d grid"""
    positions = list(range(9))
    return (
        pd.DataFrame(triples, columns=['row', 'column', 'value'])
            .pivot_table(index='row', columns='column', values='value')
            .reindex(positions)
            .reindex(positions, axis=1)
            .applymap(lambda v: str(int(v)) if v == v else '')
    )

## Creating the model

The next step is to formulate Sudoku as an optimization problem using `opvious`' [declarative modeling API](https://opvious.readthedocs.io/en/stable/modeling.html).

In [3]:
import opvious.modeling as om

class Sudoku(om.Model):
    """A mixed-integer model for Sudoku"""
    
    positions = om.interval(0, 8, name='P')
    values = om.interval(1, 9, name='V')

    def __init__(self):
        self.input = om.Parameter.indicator(self.grid * self.values, qualifiers=['row', 'column', 'value'])
        self.output = om.Variable.indicator(self.grid * self.values, qualifiers=['row', 'column', 'value'])

    @property
    def grid(self):
        """Cross-product of (row, column) positions"""
        return self.positions * self.positions

    @om.constraint
    def output_matches_input(self):
        """The output must match all input values where specified"""
        for i, j, v in self.grid * self.values:
            if self.input(i, j, v):
                yield self.output(i, j, v) >= self.input(i, j, v)

    @om.constraint
    def one_output_per_cell(self):
        """Each cell has exactly one value"""
        for i, j in self.grid:
            yield om.total(self.output(i, j, v) == 1 for v in self.values)

    @om.constraint
    def one_value_per_column(self):
        """Each value is present exactly once per column"""
        for j, v in self.positions * self.values:
            yield om.total(self.output(i, j, v) == 1 for i in self.positions)

    @om.constraint
    def one_value_per_row(self):
        """Each value is present exactly once per row"""
        for i, v in self.positions * self.values:
            yield om.total(self.output(i, j, v) == 1 for j in self.positions)

    @om.constraint
    def one_value_per_box(self):
        """Each value is present exactly once per box"""
        for v, b in self.values * self.positions:
            yield om.total(
                self.output(3 * (b // 3) + c // 3, 3 * (b % 3) + c % 3, v) == 1
                for c in self.positions
            )
            
model = Sudoku()
model.specification()  # The model's mathematical definitions

<div style="margin-top: 1em; margin-bottom: 1em;">
<details open>
<summary style="cursor: pointer; text-decoration: underline; text-decoration-style: dotted;">Sudoku</summary>
<div style="margin-top: 1em;">
$$
\begin{align*}
  \S^p_\mathrm{input[row,column,value]}&: i \in \{0, 1\}^{P \times P \times V} \\
  \S^v_\mathrm{output[row,column,value]}&: \omicron \in \{0, 1\}^{P \times P \times V} \\
  \S^a&: P \doteq \{ 0 \ldots 8 \} \\
  \S^a&: V \doteq \{ 1 \ldots 9 \} \\
  \S^c_\mathrm{outputMatchesInput}&: \forall p \in P, p' \in P, v \in V \mid i_{p,p',v} \neq 0, \omicron_{p,p',v} \geq i_{p,p',v} \\
  \S^c_\mathrm{oneOutputPerCell}&: \forall p \in P, p' \in P, \sum_{v \in V} \omicron_{p,p',v} = 1 \\
  \S^c_\mathrm{oneValuePerColumn}&: \forall p \in P, v \in V, \sum_{p' \in P} \omicron_{p',p,v} = 1 \\
  \S^c_\mathrm{oneValuePerRow}&: \forall p \in P, v \in V, \sum_{p' \in P} \omicron_{p,p',v} = 1 \\
  \S^c_\mathrm{oneValuePerBox}&: \forall v \in V, p \in P, \sum_{p' \in P} \omicron_{3 \left\lfloor \frac{p}{3} \right\rfloor + \left\lfloor \frac{p'}{3} \right\rfloor,3 \left(p \bmod 3\right) + p' \bmod 3,v} = 1 \\
\end{align*}
$$
</div>
</details>
</div>

## Finding solutions

Now that we've formulated the problem, we'll first use it to fill in Sudoku grids. We just need to pass in the initial triples as `input` parameter and [start solving](https://opvious.readthedocs.io/en/stable/overview.html#solves).

In [4]:
import opvious

client = opvious.Client.default("https://try.opvious.io")

async def fill_in(grid):
    """Completes a partial grid into a valid solution
    
    Args:
        grid: Partial grid
    """
    problem = opvious.Problem(model.specification(), parameters={'input': parse_grid(grid)})
    solution = await client.solve(problem)
    output = solution.outputs.variable('output')
    return pretty_grid(output.index.to_list())

We test that it works on an example.

In [5]:
await fill_in("""
    87.59...4
    ..2...98.
    41.7.8.25
    .5..6...2
    ....4....
    ..71.....
    .2.......
    .8.6...5.
    1.59.....
""")

column,0,1,2,3,4,5,6,7,8
row,,,,,,,,,
0,8,7,6,5,9,2,3,1,4
1,5,3,2,4,1,6,9,8,7
2,4,1,9,7,3,8,6,2,5
3,3,5,1,8,6,9,7,4,2
4,6,9,8,2,4,7,5,3,1
5,2,4,7,1,5,3,8,9,6
6,9,2,4,3,7,5,1,6,8
7,7,8,3,6,2,1,4,5,9
8,1,6,5,9,8,4,2,7,3


## Identifying mistakes

Mistakes can happen when manually solving Sudoku puzzles. We often discover these much later, making it difficult to identify which decision(s) caused the grid to become infeasible.

If we were to use `fill_in` on an infeasible grid directly, it would throw an exception. Fortunately, we only need to tweak its implementation slightly to instead detect the _smallest set of changes_ needed to restore feasibility in a grid. We simply [relax the input matching constraint](https://opvious.readthedocs.io/en/stable/transformations.html#relaxing-constraints) and output the corresponding deficit. Clearing all cells with a deficit is guaranteed to make the grid solvable again.

In [6]:
async def find_mistakes(grid):
    """Returns the smallest subgrid which restore feasibility when cleared
    
    Args:
        grid: Partial grid
    """
    problem = opvious.Problem(
        model.specification(),
        parameters={'input': parse_grid(grid)},
        transformations=[opvious.transformations.RelaxConstraints(['outputMatchesInput'])],
    )
    solution = await client.solve(problem)
    deficit = solution.outputs.variable('outputMatchesInput_deficit')
    return pretty_grid(deficit.index.to_list())

We check that it works on the infeasible grid displayed above.

In [7]:
infeasible_grid = """
    876592314
    532416987
    419738625
    351867742
    698243591
    247159863
    924385176
    783621459
    165974238
"""

await find_mistakes(infeasible_grid)

column,0,1,2,3,4,5,6,7,8
row,,,,,,,,,
0,,,,,,,,,
1,,,,,,,,,
2,,,,,,,,,
3,,5,,,,,7,,
4,,9,,,,,5,,
5,,,,,,,,,
6,,,,,,,,,
7,,,,,,,,,
8,,,,,,,,,


As a quick extension we'll allow marking certain cells as definitely correct. This can be useful to find different sets of mistakes or to prevent the solution from including the initial Sudoku numbers. For example you can notice that the 5 highlighted as mistake in row 3 was part of the infeasible grid's original input.

To do so, we just add a [transformation which pins](https://opvious.readthedocs.io/en/stable/transformations.html#pinning-variables) a configurable subset of output variables to their current value to the original `find_mistakes` implementation. This will automatically generate an `output_pin` parameter which we can use to mark certain outputs as fixed.

In [8]:
async def explore_mistakes(grid, correct=None):
    """Returns the smallest subset of input triples which restore feasibility when removed
    
    Args:
        inputs: Partial grid
        correct: List of (row, column) pairs capturing cells which are known to be correct (these
            will never be returned as mistakes)
    """
    inputs = parse_grid(grid)
    problem = opvious.Problem(
        model.specification(),
        parameters={
            'input': inputs,
            'output_pin': [t for t in inputs if t[:2] in correct] if correct else [],  # Pinned outputs
        },
        transformations=[
            opvious.transformations.RelaxConstraints(['outputMatchesInput']),
            opvious.transformations.PinVariables(['output']),  # Added transformation
        ],
    )
    solution = await client.solve(problem)
    deficit = solution.outputs.variable('outputMatchesInput_deficit')
    return pretty_grid(deficit.index.to_list())

We check that it works by marking the 5 in row 3 as correct. The function now accurately finds another way to restore feasibility.

In [9]:
await explore_mistakes(infeasible_grid, correct=[(3, 1)])

column,0,1,2,3,4,5,6,7,8
row,,,,,,,,,
0,,,,,,,,,
1,,,,,,,,,
2,,,,,,,,,
3,,,,,,7,,4,
4,,,,,4,,,9,
5,,,,,,,,,
6,,,,,,,,,
7,,,,,,,,,
8,,,,,7,4,,,
